In [ ]:
job_id=dbutils.widgets.text('job_id','')
run_id=dbutils.widgets.text('run_id','')
task_id=dbutils.widgets.text('task_id','')

In [ ]:
job_id=dbutils.widgets.get('job_id')
run_id=dbutils.widgets.get('run_id')
task_id=dbutils.widgets.get('task_id')

In [ ]:
import json

def retrieve_taskinfo_from_metadata_table(job_id,run_id,task_id):
  try:
    if job_id =='' and run_id =='' and task_id =='':
      Query = (f'''
        select 
          *
        from
          gbpdia_utils.metadata_gbp_execution m
          left join gbpdia_utils.metadata_gbp_iris_incidents i
            on m.job_id =i.job_id
            and m.run_id = i.run_id
            and m.task_id = i.task_id
        where 
          LOAD_START_DT>=now()- INTERVAL 1 DAY
          and JOB_STATUS='Failed'
          and i.incident_number is null
        ''')
    else:
      Query = (f'''
          select 
            *
          from
            gbpdia_utils.metadata_gbp_execution m
            left join gbpdia_utils.metadata_gbp_iris_incidents i
              on m.job_id =i.job_id
              and m.run_id = i.run_id
              and m.task_id = i.task_id
          where 
            'JOB_iD'='{job_id}' and 'RUN_ID'='{run_id}' and 'TASK_ID'='{task_id}'
            and JOB_STATUS='Failed'
            and i.incident_number is null
          ''')

    query_data = spark.sql(Query).collect()
    if len(query_data)>0:
      tasks_list=[]
      for row in query_data:
        #print(row)

        job_id=row['JOB_ID']
        run_id=row['RUN_ID']
        task_id=row['TASK_ID']
        
        task_info={
            
            'job_id':job_id,
            'run_id':run_id,
            'task_id':task_id,
            'task_name':row['TASK_NAME'],
            'Workspace_URL':row['WORKSPACE_URL'],
            'notebook_path':row['NOTEBOOK_PATH'],
            'result_state':row['JOB_STATUS'],
            'state_message':row['EXCEPTION']

              }

        task_info_short={
            #'job_name':job_name,
            'task_name':row['TASK_NAME'],
            'result_state':row['JOB_STATUS'],
            }
        tasks_list.append([task_info,task_info_short,job_id,run_id,task_id])
      print(tasks_list)
      print("returning task_list from metadata table")  
      return tasks_list
      
    else:
      print("No tasks found with the query search ")
        
  except Exception as Error:
          print(Error)
          print('No failed tasks found/incident already exists for this task')
    
  

    

In [ ]:
def insert_into_metadata_gbp_execution_failed_runs(job_id,run_id,task_id,incident_number):

    try:
        query=f'''insert into gbpdia_utils.metadata_gbp_iris_incidents(job_id,run_id,task_id,incident_number,insert_ts)
        values('{job_id}','{run_id}','{task_id}','{incident_number}',CURRENT_TIMESTAMP()) '''
        spark.sql(query)
    except Exception as Error:
      print(Error)
      

In [ ]:
import requests, json

def create_servicenow_incident(url,access_token_url,client_id,client_secret,resource,urgency,short_description,impact,cmdb_ci,caller_id,location,assignment_group,description,job_id,run_id,task_id):

    access_token=get_access_token(access_token_url,client_id,client_secret,resource)
   

    headers={'Authorization': f'Bearer {access_token}',
        'Content-Type':'application/json','Accept':'application/json'}

    data={
    "incident_object":{
        "urgency":urgency,
        "short_description":short_description,
        "impact":impact,
        "cmdb_ci":cmdb_ci,
        "caller_id":caller_id,
        "location": location,
        "assignment_group":assignment_group,
        "description":description
        }
    }
    response=requests.post(url,headers=headers,data=json.dumps(data))
    print(response)

    #check for http codes other than 200

    if response.status_code!=200:
        print('Status:',response.status_code,'Headers:', response.headers, 'Error Response:',response.json())
        exit()

    data=response.json()    
    incident=data['result']['return_response']['number']
    print('---------------Inserting incident data into metadata_gbp_execution_failed_runs table ')
    insert_into_metadata_gbp_execution_failed_runs(job_id,run_id,task_id,incident)
    return data



In [ ]:


def get_access_token(access_token_url,client_id,client_secret,resource):

    headers={ 'Content-Type':'application/x-www-form-urlencoded'}
    payload=  {
    'f':'pjson',
    'grant_type':'client_credentials',
    'client_id':client_id,
    'client_secret':client_secret,
    'resource':resource

          }
    
    try:

        response=requests.post(access_token_url,headers=headers,data=payload)
        if response.status_code==200:
             token=response.json().get('access_token')
             return (token)
        else:
            print(f'Failed to obtain access token. Status Code:{response.status_code}')
            print(f'Response content: {response.content.decode()}')
        
    except Exception as e:
         print(f'Error:{str(e)}') 




In [ ]:
incident_url='https://xxxxxxxxxxxxxxxxxxxxxxxx/apg-001-servicenow/v1/now/table/incident/'
access_token_url='https:/xxxxxxxxxxxxxxxxxxxxxx/oauth2/token?api-version=1.0'
databricks_url='https://xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx/api/2.1/jobs/runs/get/'
client_id='xxxxxxxxxxxxxxxxxxxxxx'
resource='xxxxxxxxxxxxxxxxxxx'
db_access_token=dbutils.secrets.get("scope_name",'secret_name')
client_secret=dbutils.secrets.get("scope_name",'secret_name')



# required to create an incident
urgency="3"
impact="2"
cmdb_ci="xxxxxxxxxx"
caller_id="123455"
location='1234'
assignment_group="xxxxxxxxxxx"

In [ ]:

task_info=retrieve_taskinfo_from_metadata_table(job_id,run_id,task_id)


In [ ]:

#print(task_info)

if task_info is not None:
    for item in task_info:
        description=json.dumps(item[0])
        short_description=json.dumps(item[1])
        job_id=item[2]
        run_id=item[3]
        task_id=item[4]
        print('--------------')
        print(job_id,run_id,task_id)
            
        print('description:',description)
        print('short description:',short_description)
        
        create_servicenow_incident(incident_url,access_token_url,client_id,client_secret,resource,urgency,short_description,impact,cmdb_ci,caller_id,location,assignment_group,description,job_id,run_id,task_id)
else:
    print('No failed tasks found or Incident already exists')
    exit()

    

